In [2]:
import os
import io
import datetime
import pandas as pd
from google.cloud import storage, bigquery
from google.cloud.exceptions import NotFound

# Initialize clients for Google Cloud Storage and BigQuery
storage_client = storage.Client()
bigquery_client = bigquery.Client()

# Set the BigQuery dataset and table
BQ_DATASET = 'sales_dataset'  # Replace with your dataset name
BQ_TABLE = 'sales_table'      # Replace with your table name

def parse_date(date_str):
    """Parse the date and return it in a standard format (ISO 8601)."""
    try:
        return datetime.datetime.strptime(date_str, '%Y-%m-%d').isoformat()
    except ValueError:
        return None  # Return None for invalid dates

def extract_data_from_gcs(bucket_name, file_name):
    """Extract the CSV file from Google Cloud Storage and load it into a pandas DataFrame."""
    # Read the CSV file from GCS
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    csv_data = blob.download_as_text()

    # Parse the CSV data into a pandas DataFrame
    df = pd.read_csv(io.StringIO(csv_data))
    print(f"Extracted {len(df)} rows from the file {file_name}.")
    return df

def transform_data(df):
    """Perform transformations on the data: handle missing values, parse dates, etc."""
    # Handle missing values (fill with default values or drop rows)
    df['quantity'].fillna(0, inplace=True)  # Fill missing quantities with 0
    df['price'].fillna(0, inplace=True)  # Ensure no null prices

    # Combine two columns into one (e.g., 'first_name' + 'last_name')
    df['full_name'] = df['first_name'] + ' ' + df['last_name']

    # Parse date fields and convert to a standard format
    df['purchase_date'] = df['purchase_date'].apply(parse_date)

    # Remove rows with invalid emails (e.g., not matching the email pattern)
    df = df[df['email'].str.contains(r'^[^@]+@[^@]+\.[^@]+$', na=False)]

    # Create a new column for total cost (quantity * price)
    df['total_cost'] = df['quantity'] * df['price']

    print("Data transformation complete.")
    return df

def load_data_to_bigquery(df, table_ref):
    """Load the transformed data into BigQuery using job configuration."""
    # Set up BigQuery job configuration with autodetect schema
    job_config = bigquery.LoadJobConfig(
        autodetect=True,  # Auto-detect the schema
        write_disposition=bigquery.WriteDisposition.WRITE_APPEND,  # Append data to the table
        source_format=bigquery.SourceFormat.CSV  # Assuming CSV, modify if necessary
    )

    # Load data into BigQuery
    try:
        load_job = bigquery_client.load_table_from_dataframe(df, table_ref, job_config=job_config)
        load_job.result()  # Wait for the job to complete
        print(f"Successfully loaded {len(df)} rows into {table_ref}.")
    except Exception as e:
        print(f"Failed to load data to BigQuery: {e}")

def load_data_to_bigquery_trigger(data, context):
    """Triggered by a change to a Cloud Storage bucket.
       This function extracts, transforms, and loads data to BigQuery.
    """
    bucket_name = data['bucket']
    file_name = data['name']

    # Log the event
    print(f"Triggered by file: gs://{bucket_name}/{file_name}")

    # Define the BigQuery table reference
    table_ref = bigquery_client.dataset(BQ_DATASET).table(BQ_TABLE)

    # 1. Extract: Get data from GCS
    df = extract_data_from_gcs(bucket_name, file_name)

    # 2. Transform: Apply transformations to the data
    transformed_df = transform_data(df)

    # 3. Load: Load the transformed data into BigQuery
    load_data_to_bigquery(transformed_df, table_ref)
